---
title: "Module 04: Lab 01"
subtitle: "Visual Reporting and Storytelling"
author: "Andrey Pafnutyev"
number-sections: true
date: "2024-11-22"
date-modified: today
date-format: long
engine: jupyter
categories: ["visualization","plotly","spark","Visual Reporting","Storytelling with Data","Industry-Specific Visualization",]
execute: 
  eval: true
  echo: true
---

# Objectives {.unnumbered}

By the end of this lab, you will:
1. Load and analyze the **Lightcast dataset** in **Spark DataFrame**.
2. Create **five easy and three medium-complexity visualizations** using **Plotly**.
3. Explore **salary distributions, employment trends, and job postings**.
4. Analyze **skills in relation to NAICS/SOC/ONET codes and salaries**.
5. Customize **colors, fonts, and styles** in all visualizations (**default themes result in a 2.5-point deduction**).
6. Follow **best practices for reporting on data communication**.

# Step 1: Load the Dataset {.unnumbered}


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "vscode"
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, collect_list, avg, max, min, count, desc, explode, split, expr


# Initialize Spark Session
spark = SparkSession.builder.appName("LightcastData").getOrCreate()

# Load Data
df = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine","true").option("escape", "\"").csv("./data/lightcast_job_postings.csv")

# Show Schema and Sample Data
df.printSchema()
df.show(5)


root
 |-- ID: string (nullable = true)
 |-- LAST_UPDATED_DATE: date (nullable = true)
 |-- LAST_UPDATED_TIMESTAMP: timestamp (nullable = true)
 |-- DUPLICATES: integer (nullable = true)
 |-- POSTED: date (nullable = true)
 |-- EXPIRED: date (nullable = true)
 |-- DURATION: integer (nullable = true)
 |-- SOURCE_TYPES: string (nullable = true)
 |-- SOURCES: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- ACTIVE_URLS: string (nullable = true)
 |-- ACTIVE_SOURCES_INFO: string (nullable = true)
 |-- TITLE_RAW: string (nullable = true)
 |-- BODY: string (nullable = true)
 |-- MODELED_EXPIRED: date (nullable = true)
 |-- MODELED_DURATION: integer (nullable = true)
 |-- COMPANY: integer (nullable = true)
 |-- COMPANY_NAME: string (nullable = true)
 |-- COMPANY_RAW: string (nullable = true)
 |-- COMPANY_IS_STAFFING: boolean (nullable = true)
 |-- EDUCATION_LEVELS: string (nullable = true)
 |-- EDUCATION_LEVELS_NAME: string (nullable = true)
 |-- MIN_EDULEVELS: integer (nullable

# Salary Distribution by Employment Type
- Identify salary trends across different employment types.
- **Filter the dataset**
  - Remove records where **salary is missing or zero**.
- **Aggregate Data**
  - Group by **employment type** and compute salary distribution.
- **Visualize results**
  - Create a **box plot** where:
    - **X-axis** = `EMPLOYMENT_TYPE_NAME`
    - **Y-axis** = `SALARY_FROM`
  - Customize **colors, fonts, and styles** to avoid a **2.5-point deduction**.
- **Explanation:** Write two sentences about what the graph reveals.



In [17]:
filtered_df = df.filter((col("SALARY").isNotNull()) & (col("SALARY") > 0))
pdf = filtered_df.select("EMPLOYMENT_TYPE_NAME", "SALARY").toPandas()

boxplot = px.box(pdf, x="EMPLOYMENT_TYPE_NAME", y="SALARY",
             title="Salary Distribution by Employment Type",
             color="EMPLOYMENT_TYPE_NAME", 
             category_orders={"EMPLOYMENT_TYPE_NAME": ["Full-time (> 32 hours)", "Part-time / full-time", "Part-time (≤ 32 hours)"]})

boxplot.update_layout(
    plot_bgcolor='white', 
    font_family="Arial",
    title_font_size=16,
    font=dict(
        size=12
    ),
    legend=dict(
        title_font_family="Arial",
        title_font_size=14,
        x=1.02,
        y=1,
        bordercolor="Black",
        borderwidth=1
    )
)

boxplot.update_xaxes(title_font=dict(size=14, family='Arial', color='grey'), tickangle=-45)
boxplot.update_yaxes(title_text="Salary ($)", title_standoff=25, gridcolor='grey')

boxplot.show()
boxplot.write_image("_output/boxplot.svg")

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


This box plot reveals the distribution of salaries across three different employment types: full-time (> 32 hours), part-time/full-time, and part-time (≤ 32 hours). It shows that full-time positions generally offer higher median salaries and a wider range of salary values compared to part-time roles, which display more compact salary distributions and lower overall salary ranges.

# Salary Distribution by Industry
- Compare salary variations across industries.
- **Filter the dataset**
  - Keep records where **salary is greater than zero**.
- **Aggregate Data**
  - Group by **NAICS industry codes**.
- **Visualize results**
  - Create a **box plot** where:
    - **X-axis** = `NAICS2_NAME`
    - **Y-axis** = `SALARY_FROM`
  - Customize colors, fonts, and styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [15]:
filtered_df = df.filter(col("SALARY_FROM") > 0)
aggregated_df = filtered_df.groupBy("NAICS2_NAME").agg(
    avg("SALARY_FROM").alias("AVG_SALARY"),
    max("SALARY_FROM").alias("MAX_SALARY"),
    min("SALARY_FROM").alias("MIN_SALARY")
)

pdf = aggregated_df.toPandas()

salarydist = px.bar(pdf, x="NAICS2_NAME", y="AVG_SALARY",
             title="Average Salary Distribution by Industry",
             labels={"NAICS2_NAME": "Industry", "AVG_SALARY": "Average Salary ($)"},
             color="AVG_SALARY")  # Color by average salary

salarydist.update_layout(
    plot_bgcolor='white',
    font_family="Arial",
    title_font_size=18
)

salarydist.show()
salarydist.write_image("_output/salarydist.svg")

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


This graph illustrates the average salary distribution across various industries like Information and Accommodation tend to offer higher average salaries. Sectors Public Administration and Educational Services have lower average salaries.

# Job Posting Trends Over Time
- Analyze how job postings fluctuate over time.
- **Aggregate Data**
  - Count job postings per **posted date (`POSTED`)**.
- **Visualize results**
  - Create a **line chart** where:
    - **X-axis** = `POSTED`
    - **Y-axis** = `Number of Job Postings`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [16]:
aggregated_df = df.groupBy("POSTED").agg(count("*").alias("NUM_POSTINGS"))
sorted_df = aggregated_df.orderBy("POSTED")
pdf = sorted_df.toPandas()

trends = px.line(pdf, x="POSTED", y="NUM_POSTINGS",
              title="Job Posting Trends Over Time",
              labels={"POSTED": "Date Posted", "NUM_POSTINGS": "Number of Job Postings"},
              line_shape="linear")

trends.update_layout(
    plot_bgcolor='white',
    font_family="Arial",
    title_font_size=18,
    font=dict(
        size=12 
    )
)

trends.update_traces(line=dict(color="#636EFA", width=2))

trends.show()
trends.write_image("_output/trends.svg")

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


This line chart shows the fluctuations in the number of job postings in 2024 between May and September, indicating a highly variable job market with peaks that suggest seasonal or periodic changes in job availability. The most significant spikes and drops highlight periods of either increased hiring activity or possible data collection anomalies, emphasizing the need for further analysis to understand these patterns better.

# Top 10 Job Titles by Count
- Identify the most frequently posted job titles.
- **Aggregate Data**
  - Count the occurrences of each **job title (`TITLE_NAME`)**.
  - Select the **top 10 most frequent titles**.
- **Visualize results**
  - Create a **bar chart** where:
    - **X-axis** = `TITLE_NAME`
    - **Y-axis** = `Job Count`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [19]:
aggregated_df = df.groupBy("TITLE_NAME").count()
top_titles_df = aggregated_df.orderBy(col("count").desc()).limit(10)
pdf = top_titles_df.toPandas()

colors = px.colors.qualitative.Vivid

toptitles = px.bar(pdf, x='TITLE_NAME', y='count',
             title='Top 10 Job Titles by Count',
             labels={'TITLE_NAME': 'Job Title', 'count': 'Job Count'},
             text='count')

toptitles.update_layout(
    plot_bgcolor='white', 
    font_family="Arial",
    title_font_size=18,
    font=dict(
        size=12, 
        color="black"
    )
)

toptitles.update_traces(marker_color='rgba(135, 206, 250, 0.8)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)

toptitles.show()
toptitles.write_image("_output/toptitles.svg")

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


This bar chart reveals the top 10 job titles by count, indicating that Data Analyst is the most frequently posted job title by a significant margin. This might underscore the critical importance of data skills in the current job market.

# Remote vs On-Site Job Postings
- Compare the proportion of remote and on-site job postings.
- **Aggregate Data**
  - Count job postings by **remote type (`REMOTE_TYPE_NAME`)**.
- **Visualize results**
  - Create a **pie chart** where:
    - **Labels** = `REMOTE_TYPE_NAME`
    - **Values** = `Job Count`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
filtered_df = df.filter((col("REMOTE_TYPE_NAME").isNotNull()) & (col("REMOTE_TYPE_NAME") != "[None]"))
aggregated_df = filtered_df.groupBy("REMOTE_TYPE_NAME").count()
pdf = aggregated_df.toPandas()

colors = ['red', 'green', 'yellow']

piechart = px.pie(pdf, names='REMOTE_TYPE_NAME', values='count',
             title='Remote vs On-Site Job Postings',
             color_discrete_sequence=colors)

piechart.update_layout(
    title_font_family="Arial",
    title_font_size=18,
    font=dict(
        family="Arial",
        size=12,
        color="black"
    )
)

piechart.show()
piechart.write_image("_output/toptitles.svg")

This pie chart shows that a vast majority of job postings (78.6%), are remote positions. Non-remote and hybrid remote job postings represent a smaller fraction, with non-remote jobs accounting for 7.13% and hybrid remote jobs making up 14.2%.


# Salary Analysis by ONET Occupation Type (Bubble Chart)
- Analyze how salaries differ across ONET occupation types.
- **Aggregate Data**
  - Compute **median salary** for each occupation in the **ONET taxonomy**.
- **Visualize results**
  - Create a **bubble chart** where:
    - **X-axis** = `ONET_NAME`
    - **Y-axis** = `Median Salary`
    - **Size** = Number of job postings
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
aggregated_df = df.groupBy("ONET_NAME").agg(
    expr("percentile_approx(SALARY, 0.5)").alias("MEDIAN_SALARY"),
    count("*").alias("JOB_COUNT")
)

salary = aggregated_df.toPandas()

salary = px.scatter(pdf, x="ONET_NAME", y="MEDIAN_SALARY",
                 size="JOB_COUNT", color="ONET_NAME",
                 hover_name="ONET_NAME", size_max=60,
                 title="Salary Analysis by ONET Occupation Type",
                 labels={"ONET_NAME": "ONET Occupation", "MEDIAN_SALARY": "Median Salary", "JOB_COUNT": "Number of Job Postings"})

salary.update_layout(
    plot_bgcolor='white', 
    font_family="Arial",
    title_font_size=18,
    font=dict(
        size=12, 
        color="black"
    )
)

salary.show()
salary.write_image("_output/salary.svg")

This bubble chart displays the ONET occupation of Business Intelligence Analysts, revealing a median salary of $116,300. The size of the bubble indicates 72476 job postings, highlighting the significant demand for this role in the job market.

# Career Pathway Trends (Sankey Diagram)
- Visualize job transitions between different occupation levels.
- **Aggregate Data**
  - Identify career transitions between **SOC job classifications**.
- **Visualize results**
  - Create a **Sankey diagram** where:
    - **Source** = `SOC_2021_2_NAME`
    - **Target** = `SOC_2021_3_NAME`
    - **Value** = Number of transitions
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
transition_count = df.groupBy("SOC_2021_2_NAME", "SOC_2021_3_NAME").count()
transition_count_pandas = transition_count.toPandas()

source_indices = {name: idx for idx, name in enumerate(transition_count_pandas['SOC_2021_2_NAME'].unique())}
target_indices = {name: idx + len(source_indices) for idx, name in enumerate(transition_count_pandas['SOC_2021_3_NAME'].unique())}

transition_count_pandas['source'] = transition_count_pandas['SOC_2021_2_NAME'].map(source_indices)
transition_count_pandas['target'] = transition_count_pandas['SOC_2021_3_NAME'].map(target_indices)

import plotly.graph_objects as go
carrerpathway = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color='black', width=0.5),
        label=list(source_indices.keys()) + list(target_indices.keys()),  
        color='purple'  
    ),
    link=dict(
        source=transition_count_pandas['source'],
        target=transition_count_pandas['target'],
        value=transition_count_pandas['count']
    )
)])

carrerpathway.update_layout(
    title_text='Career Pathway Trends',
    font=dict(size=10, color='black', family='Arial')
)

carrerpathway.show()
carrerpathway.write_image("_output/carrerpathway.svg")

This Sankey diagram reveals that there is one recorded career transition from the category of Computer and Mathematical Occupations to Mathematical Science Occupations, with no transitions recorded moving in the opposite direction or out of Mathematical Science Occupations.